# Noiseless T-CNOT

### Contructing $U$

In [120]:
import numpy as np

I2 = np.eye(2)
zero_zero = np.kron([1, 0], [1, 0])  # |00⟩
one_one  = np.kron([0, 1], [0, 1])  # |11⟩
phi_plus_ket = (zero_zero + one_one) / np.sqrt(2)  # shape (4,)
phi_plus_col = phi_plus_ket.reshape(-1, 1)  # shape (4, 1)
U = np.kron(I2, np.kron(phi_plus_col, I2))

np.set_printoptions(precision=3, suppress=True)
print("U = I2 ⊗ |Φ⁺⟩ ⊗ I2\n")
print(U)


U = I2 ⊗ |Φ⁺⟩ ⊗ I2

[[0.707 0.000 0.000 0.000]
 [0.000 0.707 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.707 0.000 0.000 0.000]
 [0.000 0.707 0.000 0.000]
 [0.000 0.000 0.707 0.000]
 [0.000 0.000 0.000 0.707]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.707 0.000]
 [0.000 0.000 0.000 0.707]]


### Contructing $V$

In [121]:
import numpy as np

def cnot_matrix(control, target, num_qubits=4):
    dim = 2 ** num_qubits
    mat = np.zeros((dim, dim))

    for i in range(dim):
        bits = list(map(int, format(i, f'0{num_qubits}b')))
        if bits[control] == 1:
            bits[target] ^= 1  # flip target
        j = int("".join(map(str, bits)), 2)
        mat[j, i] = 1

    return mat

def cz_matrix(control, target, num_qubits=4):
    """
    Construct a full CZ gate acting on `num_qubits`-qubit register,
    with control and target qubit indices (0-based).
    Diagonal matrix: entries are -1 only if both control and target bits are 1.
    """
    dim = 2 ** num_qubits
    mat = np.eye(dim)

    for i in range(dim):
        bits = format(i, f'0{num_qubits}b')  # e.g., '1010'
        ctrl_bit = int(bits[control])
        targ_bit = int(bits[target])

        if ctrl_bit == 1 and targ_bit == 1:
            mat[i, i] = -1  # apply Z phase

    return mat

def hadamard_matrix(target, num_qubits=4):
    """
    Construct an n-qubit Hadamard matrix that applies H on qubit `target`.
    Qubits are indexed from left (most significant) to right (least significant).
    """
    # Hadamard gate on one qubit
    H = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    I = np.eye(2)

    # Build list of operators for tensor product
    ops = [I] * num_qubits
    ops[target] = H  # apply H on the target qubit

    # Compute full tensor product: left to right
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)

    return result

# Pretty print
np.set_printoptions(precision=0, suppress=True, linewidth=150)


# Build CNOT with control=0 (1st qubit), target=2 (3rd qubit)
CNOT24 = cnot_matrix(control=1, target=3, num_qubits=4)
CNOT34 = cnot_matrix(control=2, target=3, num_qubits=4)
CNOT12 = cnot_matrix(control=0, target=1, num_qubits=4)
CZ13 = cz_matrix(control = 2, target = 0, num_qubits=4)
H3 = hadamard_matrix(target = 2, num_qubits=4)

V = CZ13 @ H3 @ CNOT24 @ CNOT12 @ CNOT34

np.set_printoptions(precision=3, suppress=True)
print(V)

[[ 0.707  0.000  0.000  0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.707  0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.707  0.000  0.000 -0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.707 -0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.707  0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.707  0.000  0.000  0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.707 -0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.707  0.000  0.000 -0.707  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  

### Constructing $W(x_1, x_2)$

In [122]:
def W_matrix(x1, x2):
    """
    Constructs the 4x16 matrix:
        W_{x1,x2} = I_2 ⊗ ⟨x1| ⊗ ⟨x2| ⊗ I_2
    which maps 16-dimensional (4-qubit) states to 4-dimensional output.
    """
    # Define ⟨x| as a row vector
    bra_x1 = np.array([[1, 0]]) if x1 == 0 else np.array([[0, 1]])
    bra_x2 = np.array([[1, 0]]) if x2 == 0 else np.array([[0, 1]])

    I2 = np.eye(2)

    # Build the tensor product
    result = np.kron(I2, np.kron(bra_x1, np.kron(bra_x2, I2)))  # shape: (4, 16)

    return result

# Test with x1=0, x2=1
np.set_printoptions(precision=0, suppress=True, linewidth=180)

W_00 = W_matrix(0, 0).astype(int)
W_01 = W_matrix(0, 1).astype(int)
W_10 = W_matrix(1, 0).astype(int)
W_11 = W_matrix(1, 1).astype(int)

print("\n W_00")
print(W_00)

print("\n W_01")
print(W_01)

print("\n W_10")
print(W_10)

print("\n W_11")
print(W_11)




 W_00
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]

 W_01
[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]

 W_10
[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]

 W_11
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


### Constructing $A(x_1, x_2)$

In [124]:
def A(x1, x2):
    return W_matrix(x1, x2) @ V @ U

np.set_printoptions(precision=1, suppress=True, linewidth=180)

print('\n A(0,0)')
print(A(0,0))

print('\n A(0,1)')
print(A(0,1))

print('\n A(1,0)')
print(A(1,0))

print('\n A(1,1)')
print(A(1,1))



 A(0,0)
[[0.5 0.0 0.0 0.0]
 [0.0 0.5 0.0 0.0]
 [0.0 0.0 0.0 0.5]
 [0.0 0.0 0.5 0.0]]

 A(0,1)
[[0.5 0.0 0.0 0.0]
 [0.0 0.5 0.0 0.0]
 [0.0 0.0 0.0 0.5]
 [0.0 0.0 0.5 0.0]]

 A(1,0)
[[0.5 0.0 0.0 0.0]
 [0.0 0.5 0.0 0.0]
 [0.0 0.0 0.0 0.5]
 [0.0 0.0 0.5 0.0]]

 A(1,1)
[[-0.5  0.0  0.0  0.0]
 [ 0.0 -0.5  0.0  0.0]
 [ 0.0  0.0  0.0 -0.5]
 [ 0.0  0.0 -0.5  0.0]]


In [135]:
import numpy as np

def unique_up_to_scalar(matrices, tol=1e-8):
    """
    Given a list of matrices, returns one representative from each scalar-equivalence class.
    Two matrices A and B are considered equivalent if B = α * A for some nonzero scalar α,
    including negative or complex scalars.

    Parameters:
    ----------
    matrices : list of np.ndarray
        List of matrices to group by scalar equivalence.
    tol : float
        Tolerance used for floating point comparisons (default: 1e-8).

    Returns:
    -------
    list of np.ndarray
        List of representative matrices, one for each scalar equivalence class.
    """
    def normalize(mat):
        norm = np.linalg.norm(mat)
        if norm < tol:
            return np.zeros_like(mat)
        return mat / norm

    def is_equivalent(A, B):
        if A.shape != B.shape:
            return False
        A_norm = normalize(A)
        B_norm = normalize(B)
        # Check both direct and negated equivalence
        return (
            np.allclose(A_norm, B_norm, atol=tol) or
            np.allclose(A_norm, -B_norm, atol=tol)
        )

    unique = []
    for mat in matrices:
        if not any(is_equivalent(mat, u) for u in unique):
            unique.append(mat)

    return unique

matrices = [A(0,0), A(0,1), A(1,0), A(1,1)]

unique_up_to_scalar(matrices)[0]

array([[0.5, 0.0, 0.0, 0.0],
       [0.0, 0.5, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.5],
       [0.0, 0.0, 0.5, 0.0]])

There is only one unique matrix here which mean there is only one $A$ in the equation $\displaystyle \mathcal{M}(\rho) = \sum_{k=1}^{m} A_k\,\rho\,A_k^\dagger$

$A = \frac{1}{2} CNOT_{1,4}$ 

# Noisy TCNOT (One Amplitude Damping channel in the 2nd qubit)

### Keeping V and W the same

In [138]:
import numpy as np

def cnot_matrix(control, target, num_qubits=4):
    dim = 2 ** num_qubits
    mat = np.zeros((dim, dim))

    for i in range(dim):
        bits = list(map(int, format(i, f'0{num_qubits}b')))
        if bits[control] == 1:
            bits[target] ^= 1  # flip target
        j = int("".join(map(str, bits)), 2)
        mat[j, i] = 1

    return mat

def cz_matrix(control, target, num_qubits=4):
    """
    Construct a full CZ gate acting on `num_qubits`-qubit register,
    with control and target qubit indices (0-based).
    Diagonal matrix: entries are -1 only if both control and target bits are 1.
    """
    dim = 2 ** num_qubits
    mat = np.eye(dim)

    for i in range(dim):
        bits = format(i, f'0{num_qubits}b')  # e.g., '1010'
        ctrl_bit = int(bits[control])
        targ_bit = int(bits[target])

        if ctrl_bit == 1 and targ_bit == 1:
            mat[i, i] = -1  # apply Z phase

    return mat

def hadamard_matrix(target, num_qubits=4):
    """
    Construct an n-qubit Hadamard matrix that applies H on qubit `target`.
    Qubits are indexed from left (most significant) to right (least significant).
    """
    # Hadamard gate on one qubit
    H = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    I = np.eye(2)

    # Build list of operators for tensor product
    ops = [I] * num_qubits
    ops[target] = H  # apply H on the target qubit

    # Compute full tensor product: left to right
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)

    return result

# Pretty print
np.set_printoptions(precision=0, suppress=True, linewidth=150)


# Build CNOT with control=0 (1st qubit), target=2 (3rd qubit)
CNOT24 = cnot_matrix(control=1, target=3, num_qubits=4)
CNOT34 = cnot_matrix(control=2, target=3, num_qubits=4)
CNOT12 = cnot_matrix(control=0, target=1, num_qubits=4)
CZ13 = cz_matrix(control = 2, target = 0, num_qubits=4)
H3 = hadamard_matrix(target = 2, num_qubits=4)

V = CZ13 @ H3 @ CNOT24 @ CNOT12 @ CNOT34

def W_matrix(x1, x2):
    """
    Constructs the 4x16 matrix:
        W_{x1,x2} = I_2 ⊗ ⟨x1| ⊗ ⟨x2| ⊗ I_2
    which maps 16-dimensional (4-qubit) states to 4-dimensional output.
    """
    # Define ⟨x| as a row vector
    bra_x1 = np.array([[1, 0]]) if x1 == 0 else np.array([[0, 1]])
    bra_x2 = np.array([[1, 0]]) if x2 == 0 else np.array([[0, 1]])

    I2 = np.eye(2)

    # Build the tensor product
    result = np.kron(I2, np.kron(bra_x1, np.kron(bra_x2, I2)))  # shape: (4, 16)

    return result

W_00 = W_matrix(0, 0).astype(int)
W_01 = W_matrix(0, 1).astype(int)
W_10 = W_matrix(1, 0).astype(int)
W_11 = W_matrix(1, 1).astype(int)



### Amplitude Damping Noise in U

In [139]:
import numpy as np

def generate_Us(r):
    """
    Given a noise parameter r (0 <= r <= 1), returns:
      - N1, N2: single-qubit Kraus operators (2×2)
      - U1, U2: 4-qubit isometries (16×4) embedding (N1⊗I)|Φ+> and (N2⊗I)|Φ+>
    
    Parameters:
    -----------
    r : float
        Noise parameter between 0 and 1.

    Returns:
    --------
    N1, N2 : np.ndarray
        Single-qubit operators:
            N1 = [[1, 0], [0, sqrt(1-r)]]
            N2 = [[0, sqrt(r)], [0, 0]]
    U1, U2 : np.ndarray
        4-qubit matrices of shape (16, 4):
            U1 = I2 ⊗ (N1⊗I2)|Φ+> ⊗ I2
            U2 = I2 ⊗ (N2⊗I2)|Φ+> ⊗ I2
    """
    # Single-qubit identities and Kraus operators
    I2 = np.eye(2)
    N1 = np.array([[1, 0],
                   [0, np.sqrt(1 - r)]])
    N2 = np.array([[0, np.sqrt(r)],
                   [0, 0]])
    
    # Bell state |Φ+> = (|00> + |11>) / sqrt(2)
    phi_plus = (np.kron([1, 0], [1, 0]) + np.kron([0, 1], [0, 1])) / np.sqrt(2)
    phi_plus_col = phi_plus.reshape(-1, 1)  # shape (4,1)
    
    # Apply (Nk ⊗ I2) to |Φ+>
    v1 = np.kron(N1, I2) @ phi_plus_col
    v2 = np.kron(N2, I2) @ phi_plus_col
    
    # Construct U1 and U2: I2 ⊗ vk ⊗ I2
    U1 = np.kron(I2, np.kron(v1, I2))
    U2 = np.kron(I2, np.kron(v2, I2))
    
    return N1, N2, U1, U2

np.set_printoptions(precision=3, suppress=True, linewidth=150)
r = 0.5
N1, N2, U1, U2 = generate_Us(r)
np.set_printoptions(precision=3, suppress=True)
print("For r =", r)
print("N1 =\n", N1)
print("N2 =\n", N2)
print("U1:\n", U1)
print("U2:\n", U2)



For r = 0.5
N1 =
 [[1.000 0.000]
 [0.000 0.707]]
N2 =
 [[0.000 0.707]
 [0.000 0.000]]
U1:
 [[0.707 0.000 0.000 0.000]
 [0.000 0.707 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.500 0.000 0.000 0.000]
 [0.000 0.500 0.000 0.000]
 [0.000 0.000 0.707 0.000]
 [0.000 0.000 0.000 0.707]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.500 0.000]
 [0.000 0.000 0.000 0.500]]
U2:
 [[0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.500 0.000 0.000 0.000]
 [0.000 0.500 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.500 0.000]
 [0.000 0.000 0.000 0.500]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000]]


### Constructing $A$

In [144]:
def A_noise(x1, x2, U_):
    return W_matrix(x1, x2) @ V @ U_

np.set_printoptions(precision=3, suppress=True, linewidth=180)

W_00VU1 = A_noise(0,0,U1)
W_00VU2 = A_noise(0,0,U2)
W_01VU1 = A_noise(0,1,U1)
W_01VU2 = A_noise(0,1,U2)
W_10VU1 = A_noise(1,0,U1)
W_10VU2 = A_noise(1,0,U2)
W_11VU1 = A_noise(1,1,U1)
W_11VU2 = A_noise(1,1,U2)



[[0.500 0.000 0.000 0.000]
 [0.000 0.500 0.000 0.000]
 [0.000 0.000 0.000 0.354]
 [0.000 0.000 0.354 0.000]]


In [143]:
def unique_up_to_scalar(matrices, tol=1e-8):
    """
    Given a list of matrices, returns one representative from each scalar-equivalence class.
    Two matrices A and B are considered equivalent if B = α * A for some nonzero scalar α,
    including negative or complex scalars.

    Parameters:
    ----------
    matrices : list of np.ndarray
        List of matrices to group by scalar equivalence.
    tol : float
        Tolerance used for floating point comparisons (default: 1e-8).

    Returns:
    -------
    list of np.ndarray
        List of representative matrices, one for each scalar equivalence class.
    """
    def normalize(mat):
        norm = np.linalg.norm(mat)
        if norm < tol:
            return np.zeros_like(mat)
        return mat / norm

    def is_equivalent(A, B):
        if A.shape != B.shape:
            return False
        A_norm = normalize(A)
        B_norm = normalize(B)
        # Check both direct and negated equivalence
        return (
            np.allclose(A_norm, B_norm, atol=tol) or
            np.allclose(A_norm, -B_norm, atol=tol)
        )

    unique = []
    for mat in matrices:
        if not any(is_equivalent(mat, u) for u in unique):
            unique.append(mat)

    return unique

matrices = [W_00VU1, W_00VU2, W_01VU1, W_01VU2, W_10VU1, W_10VU2, W_11VU1, W_11VU2]

unique_up_to_scalar(matrices)

[array([[0.500, 0.000, 0.000, 0.000],
        [0.000, 0.500, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.354],
        [0.000, 0.000, 0.354, 0.000]]),
 array([[0.000, 0.354, 0.000, 0.000],
        [0.354, 0.000, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.000]]),
 array([[0.354, 0.000, 0.000, 0.000],
        [0.000, 0.354, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.500],
        [0.000, 0.000, 0.500, 0.000]]),
 array([[0.000, 0.000, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.000],
        [0.000, 0.000, 0.354, 0.000],
        [0.000, 0.000, 0.000, 0.354]])]

These are the four \(A\) matrices when \(r = 0.5\) in the amplitude-damping channel (see PDF).

The amplitude-damping channel Kraus operators are

$$
N_0 = \begin{pmatrix}
1 & 0 \\[6pt]
0 & \sqrt{1-r}
\end{pmatrix},
\qquad
N_1 = \begin{pmatrix}
0 & \sqrt{r} \\[6pt]
0 & 0
\end{pmatrix}.
$$

At \(r=0.5\), these become

$$
N_0 = \begin{pmatrix}
1 & 0 \\[6pt]
0 & \sqrt{0.5}
\end{pmatrix},
\qquad
N_1 = \begin{pmatrix}
0 & \sqrt{0.5} \\[6pt]
0 & 0
\end{pmatrix}.
$$

1.  
$$
A_{1} = 
\begin{pmatrix}
0.500 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.500 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.354 \\[6pt]
0.000 & 0.000 & 0.354 & 0.000
\end{pmatrix}
$$

2.  
$$
A_{2} = 
\begin{pmatrix}
0.000 & 0.354 & 0.000 & 0.000 \\[6pt]
0.354 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.000
\end{pmatrix}
$$

3.  
$$
A_{3} = 
\begin{pmatrix}
0.354 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.354 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.500 \\[6pt]
0.000 & 0.000 & 0.500 & 0.000
\end{pmatrix}
$$

4.  
$$
A_{4} = 
\begin{pmatrix}
0.000 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.000 \\[6pt]
0.000 & 0.000 & 0.354 & 0.000 \\[6pt]
0.000 & 0.000 & 0.000 & 0.354
\end{pmatrix}
$$
